In [6]:
import pandas as pd
import os
pd.options.display.max_columns = 200

# Import Dati Sorgente
Vengono importate le seguenti sorgenti dati:
1. CUP con feedback dal DIPE e relativi template associati
2. CUP scaricati da OpenCoesione [(vedi qui - link)](https://opencoesione.gov.it/it/progetti/?q=&selected_facets=is_pubblicato:0&selected_facets=is_pubblicato:1&selected_facets=is_pubblicato:2)

In [7]:
#  xls_DIPE = pd.ExcelFile('../result/CUP_feedback.xlsx')
xls_DIPE = pd.ExcelFile('../data/01_raw/Tutti i CUP al 20230322.xlsx')
df = pd.read_excel(xls_DIPE, 'CUP ').iloc[:-2,:]
print(df.info())
df.to_parquet('../data/02_primary/CUP_feedback.xlsx')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60343 entries, 0 to 60342
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CODICE_CUP             60343 non-null  object        
 1   SOGGETTO_TITOLARE      60343 non-null  object        
 2   DATA_GENERAZIONE       60341 non-null  datetime64[ns]
 3   TEMPLATE               60343 non-null  float64       
 4   COPERTURA_FINANZIARIA  60343 non-null  object        
 5   COSTO                  60343 non-null  float64       
 6   FINANZIAMENTO          60343 non-null  float64       
 7   DESCRIZIONE_AGGREGATA  60343 non-null  object        
 8   NATURA                 60343 non-null  object        
 9   TIPOLOGIA              60343 non-null  object        
 10  SETTORE                60343 non-null  object        
 11  SOTTOSETTORE           60343 non-null  object        
 12  CATEGORIA              60343 non-null  object        
 13  S

In [8]:
DIRNAME = '../data/01_raw/Lista CUP'

dfs = []
lenghts = []
for filename in os.listdir('../data/01_raw/Lista CUP'):
    if filename.startswith('daCUP'):
        with pd.ExcelFile(os.path.join(DIRNAME, filename)) as fl:
            data = pd.read_excel(fl,'CUP').copy()
            dfs.append(data)
            lenghts.append(len(data))
df = pd.concat(dfs, axis = 0, ignore_index=True).drop_duplicates()
df.to_csv('../data/02_primary/CUP_feedback.parquet', index=False, quoting=1)

In [9]:
df.CODICE_CUP.to_clipboard()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26498 entries, 0 to 26530
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CODICE_CUP             26498 non-null  object 
 1   SOGGETTO_TITOLARE      26498 non-null  object 
 2   DATA_GENERAZIONE       26497 non-null  object 
 3   TEMPLATE               25781 non-null  float64
 4   COPERTURA_FINANZIARIA  26498 non-null  object 
 5   COSTO                  26498 non-null  int64  
 6   FINANZIAMENTO          26498 non-null  int64  
 7   DESCRIZIONE_AGGREGATA  26498 non-null  object 
 8   NATURA                 26498 non-null  object 
 9   TIPOLOGIA              26498 non-null  object 
 10  SETTORE                26498 non-null  object 
 11  SOTTOSETTORE           26498 non-null  object 
 12  CATEGORIA              26498 non-null  object 
 13  STATO_PROGETTO         26498 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 3.0+

In [11]:

df_OC = pd.read_csv(
    '../data/01_raw/progetti_OC.zip',
    compression='zip', 
    sep=';',
    quotechar='"', 
    low_memory=False, 
    usecols=['COD_LOCALE_PROGETTO','CUP','OC_TITOLO_PROGETTO','OC_DESCR_CICLO','OC_COD_CICLO','OC_TEMA_SINTETICO','FONDO_COMUNITARIO','OC_CODICE_PROGRAMMA','OC_DESCRIZIONE_PROGRAMMA','COD_STRUMENTO','DESCR_STRUMENTO','OC_STATO_FINANZIARIO','OC_STATO_PROGETTO','OC_STATO_PROCEDURALE'],
    header=0,
    #nrows=10
    )

In [12]:
df.CODICE_CUP

0        B11C22001140006
1        B11F22002130006
2        B11F22002140006
3        B11F22002150006
4        B11F22002160006
              ...       
26526    J91F22003400006
26527    J91F22003530006
26528    J91F22003540006
26529    J91F22003580006
26530    J91F22003590006
Name: CODICE_CUP, Length: 26498, dtype: object

In [38]:
df_pad2026 = pd.read_csv('../data/01_raw/LISTA_CUP_SF.csv') # estratto direttamente da Salesforce tramite query
# df_pad2026 = pd.read_csv('../data/01_raw/CUP_output_20230130.csv')
df_pad2026.columns = ['_','CODICE_CUP','__','AVVISO','STATO_CANDIDATURA']
df_pad2026.drop(columns=['_','__'],  inplace = True)
df_pad2026.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26575 entries, 0 to 26574
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CODICE_CUP         26574 non-null  object
 1   AVVISO             26575 non-null  object
 2   STATO_CANDIDATURA  26575 non-null  object
dtypes: object(3)
memory usage: 623.0+ KB


In [39]:
df_pad2026.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26575 entries, 0 to 26574
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CODICE_CUP         26574 non-null  object
 1   AVVISO             26575 non-null  object
 2   STATO_CANDIDATURA  26575 non-null  object
dtypes: object(3)
memory usage: 623.0+ KB


In [40]:
candidature_attive = df_pad2026.STATO_CANDIDATURA.isin(['ACCETTATA','FINANZIATA'])
CUP_DUPLICATI = df_pad2026[candidature_attive].CODICE_CUP.duplicated()
df_pad2026[candidature_attive][CUP_DUPLICATI]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA
10787,D31F22004420006,"1.4.5 ""Piattaforma Notifiche Digitali” Comuni ...",FINANZIATA
20516,D91F22003340006,1.2 Abilitazione al Cloud - Comuni - Luglio 2022,FINANZIATA
21777,F91F22002170006,1.4.3 - pagoPA - Comuni - Aprile 2022,FINANZIATA


# Controlli da fare

Di seguito vengono fatti i seguenti controlli, in ordine di priorità:

1. CUP PNRR in OpenCoesione
2. CUP duplicato nel PNRR
3. CUP non esistente nei sistemi DIPE
4. CUP senza template
5. CUP template diverso


In [41]:
# concateno dataset da OpenCoesione (OC)
# df_OC = pd.concat([df_OC, df_OC2]).copy()

df_OC = df_OC.drop_duplicates(subset='CUP',keep='last').copy()

In [42]:
print(df_OC.CUP.nunique())

1274053


In [43]:
df_DIPE = pd.merge(df_pad2026,df.drop_duplicates(),  on = 'CODICE_CUP', how ='left').drop_duplicates()
df_DIPE.loc[df_DIPE.STATO_PROGETTO.isna(),'STATO_PROGETTO'] = 'CUP INESISTENTE'

In [44]:
df_DIPE['CUP'] = df_DIPE.CODICE_CUP

In [45]:
df_DIPE[df_DIPE.TEMPLATE.notna()].TEMPLATE

0        2203007.0
1        2203007.0
2        2203006.0
3        2208001.0
4        2208001.0
           ...    
26570    2203007.0
26571    2203006.0
26572    2210002.0
26573    2203006.0
26574    2203004.0
Name: TEMPLATE, Length: 25788, dtype: float64

In [46]:
# Primi check


## CUP double funding
DOUBLE_FUNDING_CUP = pd.merge( df_OC['CUP'], df_DIPE['CUP'], on = 'CUP',how='inner',validate="1:m").CUP.unique()
## CUP double project
DOUBLE_REQUEST_CUP = df_DIPE.groupby(by = 'CUP').filter(lambda x: len(x)>1).sort_values(by='CUP').CUP.unique()
## CUP non esistente nei sistemi DIPE
MISSING_CUP = df_DIPE[df_DIPE.STATO_PROGETTO=='CUP INESISTENTE'].CUP.unique()
## Template mancante
MISSING_TEMPLATE = df_DIPE[df_DIPE.TEMPLATE.isna()].CUP.unique()
#Template errato
map_sheet2template = df_DIPE[(df_DIPE.TEMPLATE.notna())].groupby(by='AVVISO').TEMPLATE.agg(pd.Series.mode).to_dict()
# map_sheet2template['1.4.4 Altri Enti FINESTRA_76'] = 20220511
# map_sheet2template['1.4.3 Altri Enti App IOFINESTRA'] = 20220509
# map_sheet2template[ '1.4.3 pagoPA Altri EntiFINESTRA'] = 20220510
WRONG_TEMPLATE = [] 
WRONG_TEMPLATE = df_DIPE[df_DIPE.AVVISO.replace(map_sheet2template) != df_DIPE.TEMPLATE].CUP.unique()

In [47]:
def classify_CUP(CUP: str):
    '''
    Funzione per produrre una semplice classificazione dei CUP secondo una scala di priorità:
    1. CUP PNRR in OpenCoesione
    2. CUP duplicato nel PNRR
    3. CUP non esistente nei sistemi DIPE
    4. CUP senza template
    5. CUP template diverso
    '''

    if CUP in DOUBLE_FUNDING_CUP:
        return 'CUP già presente in OPEN COESIONE'
    elif CUP in MISSING_CUP:
        return 'CUP NON PRESENTE NEL FEEDBACK DIPE'
    elif CUP in DOUBLE_REQUEST_CUP:
        return "CUP PRESENTE IN PIU' CANDIDATURE"
    elif CUP in MISSING_TEMPLATE:
        return 'TEMPLATE MANCANTE'
    elif CUP in WRONG_TEMPLATE:
        return 'TEMPLATE ERRATO'
    else:
        return 'NESSUN PROBLEMA RISCONTRATO AL MOMENTO'

In [48]:
# applica classificazione
df_DIPE['CLASSIFICAZIONE_ISSUE_CUP']  = df_DIPE.CUP.apply(classify_CUP)

In [49]:
print(df_DIPE.CUP.count())
#crea aggregati
df_DIPE.groupby(by=['CLASSIFICAZIONE_ISSUE_CUP']).CUP.count()

26573


CLASSIFICAZIONE_ISSUE_CUP
CUP NON PRESENTE NEL FEEDBACK DIPE           81
CUP PRESENTE IN PIU' CANDIDATURE             66
NESSUN PROBLEMA RISCONTRATO AL MOMENTO    25357
TEMPLATE ERRATO                             367
TEMPLATE MANCANTE                           702
Name: CUP, dtype: int64

In [52]:
df_DIPE[df_DIPE.CLASSIFICAZIONE_ISSUE_CUP=="CUP NON PRESENTE NEL FEEDBACK DIPE"][['CODICE_CUP','AVVISO','STATO_CANDIDATURA']].to_clipboard()

In [ ]:
df_DIPE[df_DIPE.CUP.isin(['F91F22002170006','D91F22003340006'])]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO,CUP,CLASSIFICAZIONE_ISSUE_CUP
2000,F91F22002170006,1.4.3 - pagoPA - Comuni - Settembre 2022,FINANZIATA,COMUNE DI SANTA GIUSTINA - BELLUNO -,2022-08-05 00:00:00,2203007.0,COMUNITARIA,7713.0,7713.0,PIATTAFORMA PAGOPA*TERRITORIO NAZIONALE*ATTIVA...,ACQUISTO O REALIZZAZIONE DI SERVIZI,APPLICATIVI E PIATTAFORME WEB,SERVIZI PER LA P.A. E PER LA COLLETTIVITA',SERVIZI E APPLICAZIONI INFORMATICHE PER I CITT...,SERVIZI ED APPLICAZIONI PER IL PUBBLICO,Attivo,F91F22002170006,CUP PRESENTE IN PIU' CANDIDATURE
7868,D91F22003340006,1.4.1 Esperienza del Cittadino - Comuni - Sett...,FINANZIATA,COMUNE DI CASIRATE D'ADDA - BERGAMO -,2022-10-27 00:00:00,2204004.0,COMUNITARIA,79922.0,79922.0,MIGLIORAMENTO DELLESPERIENZA DUSO DEL SITO E...,ACQUISTO O REALIZZAZIONE DI SERVIZI,APPLICATIVI E PIATTAFORME WEB,SERVIZI PER LA P.A. E PER LA COLLETTIVITA',SERVIZI E APPLICAZIONI INFORMATICHE PER I CITT...,SERVIZI ED APPLICAZIONI PER IL PUBBLICO,Attivo,D91F22003340006,CUP PRESENTE IN PIU' CANDIDATURE
20516,D91F22003340006,1.2 Abilitazione al Cloud - Comuni - Luglio 2022,FINANZIATA,COMUNE DI CASIRATE D'ADDA - BERGAMO -,2022-10-27 00:00:00,2204004.0,COMUNITARIA,79922.0,79922.0,MIGLIORAMENTO DELLESPERIENZA DUSO DEL SITO E...,ACQUISTO O REALIZZAZIONE DI SERVIZI,APPLICATIVI E PIATTAFORME WEB,SERVIZI PER LA P.A. E PER LA COLLETTIVITA',SERVIZI E APPLICAZIONI INFORMATICHE PER I CITT...,SERVIZI ED APPLICAZIONI PER IL PUBBLICO,Attivo,D91F22003340006,CUP PRESENTE IN PIU' CANDIDATURE
21777,F91F22002170006,1.4.3 - pagoPA - Comuni - Aprile 2022,FINANZIATA,COMUNE DI SANTA GIUSTINA - BELLUNO -,2022-08-05 00:00:00,2203007.0,COMUNITARIA,7713.0,7713.0,PIATTAFORMA PAGOPA*TERRITORIO NAZIONALE*ATTIVA...,ACQUISTO O REALIZZAZIONE DI SERVIZI,APPLICATIVI E PIATTAFORME WEB,SERVIZI PER LA P.A. E PER LA COLLETTIVITA',SERVIZI E APPLICAZIONI INFORMATICHE PER I CITT...,SERVIZI ED APPLICAZIONI PER IL PUBBLICO,Attivo,F91F22002170006,CUP PRESENTE IN PIU' CANDIDATURE


In [ ]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP NON PRESENTE NEL FEEDBACK DIPE")&(df_DIPE.STATO_CANDIDATURA=='FINANZIATA')].CODICE_CUP.drop_duplicates().to_csv('../data/02_primary/missing_CUP.csv', index=False, quoting=1)

In [ ]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP PRESENTE IN PIU' CANDIDATURE")&(df_DIPE.STATO_CANDIDATURA=='FINANZIATA')].CUP.value_counts()>1
# GIà SANATO
# eventualmente approfondire

F91F22002170006     True
D91F22003340006     True
E31C22001360006    False
J41F22002450006    False
J21F22002490006    False
J11F22002310006    False
B51F22002130006    False
B91C22001040006    False
B61C22001070006    False
D51F22002870006    False
G21F22002260006    False
C81F22002280006    False
E91F22002250006    False
G61F22002770006    False
I31F22002770006    False
B81F22002740006    False
D81F22002430006    False
F61F22002080006    False
I71F22002100006    False
D81F22002440006    False
F61F22002320006    False
J41F22002440006    False
B51F22002330006    False
B71F22002310006    False
H61C22001120006    False
H41C22000980006    False
J71C22000960006    False
I57H22001840001    False
C51F22002010006    False
H61C22001090006    False
J51F22002080006    False
Name: CUP, dtype: bool

In [ ]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP già presente in OPEN COESIONE")].SOGGETTO_TITOLARE.to_list()
# 

[]

In [ ]:
df_DIPE[df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP già presente in OPEN COESIONE"]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO,CUP,CLASSIFICAZIONE_ISSUE_CUP


In [ ]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP=='CUP NON PRESENTE NEL FEEDBACK DIPE')&(df_DIPE.STATO_CANDIDATURA!='RINUNCIATA')].CUP.to_list()

['E61F22004310006',
 'J41F22003350006',
 'I51C22001150006',
 'D51F22007190006',
 'D21F23000260006',
 'E41C22001540006',
 'H81F22004610006',
 'B51F22002530006',
 'D41F22002920006',
 'G61F22003110006',
 'I81F22004010006',
 'B61F22003000002',
 'H19I22000880001',
 'H81F22004600006',
 'B41C22002280006',
 'J41F22004840006',
 'H81F22003860007',
 'G41F22004570006',
 'F91F22002360006',
 'I51F22003630006',
 'I31F22005280006',
 'C31F22002570006',
 'C31C22001180006',
 'I21C22001420006',
 'D81F22004820006',
 'E51F22003880006',
 'H71J22000220006',
 'B11F22003530006',
 'H11F22004140006',
 'C41F22004810006',
 'J51F22008750006',
 'D81F22004400006',
 'D81C22002790006',
 'B91I22000290006',
 'E31F22003930006',
 'I81F22004020006',
 'B31C22001910006',
 'G31F22004900006',
 'G31C22002120006',
 'E51F22006450006',
 'E11F22005090006',
 'J51F22003810006',
 'I71F22004390001',
 'E51F22008790006',
 'F71F22002250006',
 'G61F22005190006',
 'F71F22002260006',
 'J81F22002230006',
 'I97H22002270006',
 'J71F22002950006',


In [ ]:
df[df['CODICE_CUP']=='C71C22001500001']

,CODICE_CUP,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO
772,C71C22001500001,COMUNE DI CISERANO - BERGAMO -,2022-10-14 00:00:00,NaN,STATALE,32589,32589,"COMUNE DI CISERANO*VIA P.ZZA GIOVANNI XXIII, N...",ACQUISTO O REALIZZAZIONE DI SERVIZI,APPLICATIVI E PIATTAFORME WEB,SERVIZI PER LA P.A. E PER LA COLLETTIVITA',SERVIZI E TECNOLOGIE PER L'INFORMAZIONE E LE C...,INFORMAZIONE E COMUNICAZIONE PER ATTIVITA' IST...,Attivo


In [ ]:
df_DIPE.to_csv('../data/02_primary/CUP_analysis.csv', sep=";", quoting=1, index=False)

In [ ]:
df_DIPE.CLASSIFICAZIONE_ISSUE_CUP==''

0        False
1        False
2        False
3        False
4        False
         ...  
26570    False
26571    False
26572    False
26573    False
26574    False
Name: CLASSIFICAZIONE_ISSUE_CUP, Length: 26574, dtype: bool

In [ ]:
output = pd.read_csv('../data/01_raw/CUP_output_20230130.csv')
output.CODICE_CUP.nunique()

26553

In [ ]:
len(set(set(output.CODICE_CUP.unique())).difference(df_DIPE.CODICE_CUP.unique()))

81

In [ ]:
output.CODICE_CUP.nunique() - 81

26472

In [ ]:
output.CODICE_CUP.nunique()

26553

In [ ]:
df_DIPE[df_DIPE.CODICE_CUP.isna()]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO,CUP,CLASSIFICAZIONE_ISSUE_CUP
17278,NaN,1.3.1 Piattaforma Digitale Nazionale Dati - Co...,FINANZIATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,NaN,NESSUN PROBLEMA RISCONTRATO AL MOMENTO


In [ ]:
df_DIPE.CODICE_CUP.nunique()

26540

In [ ]:
output.

,CODICE_CUP,DATA RICEZIONE CUP,ENTE,AVVISO,MISURA,FINESTRA TEMPORALE,DATA INIZIO FINESTRA,DATA FINE FINESTRA
0,G91F22002170006,2022-08-08,Comune di Castiglione della Pescaia,1.4.3 - pagoPA - Comuni - Aprile 2022,1.4.3 Adozione PagoPA e AppIO,FINESTRA_21,2022-07-05,2022-08-03
1,J61F22004400006,2022-11-29,Comune di Girifalco,1.4.3 - pagoPA - Comuni - Settembre 2022,1.4.3 Adozione PagoPA e AppIO,FINESTRA_97,2022-09-12,2022-11-25
2,F71F22004530006,2022-11-30,Comune di Fogliano Redipuglia,1.4.3 - app IO - Comuni - Settembre 2022,1.4.3 Adozione PagoPA e AppIO,FINESTRA_94,2022-09-12,2022-11-25
3,C41F22004270006,2022-11-15,Comune di Soleto,"1.4.5 ""Piattaforma Notifiche Digitali” Comuni ...",1.4.5 Digitalizzazione degli avvisi pubblici,FINESTRA_93,2022-10-12,2022-11-11
4,G91F22003850006,2022-11-17,Comune di Acquasparta,"1.4.5 ""Piattaforma Notifiche Digitali” Comuni ...",1.4.5 Digitalizzazione degli avvisi pubblici,FINESTRA_93,2022-10-12,2022-11-11
...,...,...,...,...,...,...,...,...
26583,I21F22004280006,2022-11-29,Comune di Gualdo Tadino,1.4.3 - pagoPA - Comuni - Settembre 2022,1.4.3 Adozione PagoPA e AppIO,FINESTRA_97,2022-09-12,2022-11-25
26584,C11F22004790006,2022-11-29,Comune di Vaglio Serra,1.4.3 - app IO - Comuni - Settembre 2022,1.4.3 Adozione PagoPA e AppIO,FINESTRA_94,2022-09-12,2022-11-25
26585,F51F22006420006,2022-12-23,Comune di Racalmuto,1.3.1 Piattaforma Digitale Nazionale Dati - Co...,1.3.1 Piattaforma Digitale Nazionale Dati,FINESTRA_107,2022-10-20,2022-12-14
26586,E41F22002830006,2022-09-12,Comune di Malgrate,1.4.3 - app IO - Comuni - Aprile 2022,1.4.3 Adozione PagoPA e AppIO,FINESTRA_17,2022-08-04,2022-09-02
